In [1]:
import pandas as pd

In [2]:
import xgboost as xgb
model_xgb = xgb.XGBRegressor()
model_xgb.load_model('XGB_normalized_top5_model.json')

In [3]:
def FaceLandmarksPreprocessing(df_data, verbose = False):
    for col_name in df_data.columns.to_list():
        if col_name.find('FaceLandmarks_') == 0 or \
        col_name.find('EyeLandmarks_') == 0 or \
        col_name.find('LeftEyeBoundingBox_') == 0 or \
        col_name.find('RightEyeBoundingBox_') == 0 or \
        col_name.find('LeftEyeMidPoint_') == 0 or \
        col_name.find('RightEyeMidPoint_') == 0:
            if col_name.find('_X') == len(col_name) - 2:
                df_data[col_name] = df_data[col_name] - df_data['FaceBoundingBox_X']
                if verbose:
                    print(f'Column {col_name} is subtracted by FaceBoundingBox_X.')
            elif col_name.find('_Y') == len(col_name) - 2:
                df_data[col_name] = df_data[col_name] - df_data['FaceBoundingBox_Y']
                if verbose:
                    print(f'Column {col_name} is subtracted by FaceBoundingBox_Y.')
            else:
#                 print(f'Ignore column {col_name}.')
                pass
        else:
#             print(f'Ignore column {col_name}')
            pass
    return df_data

# 列出重要的特徵。
## 正相關特徵
### top n
col_positive = ['HeadPoseAngles_Y', 'GazeVector_X', 'GazeVector_Z', 'GazeVector_Y', 'HeadPoseAngles_Z']
### 6pos6neg_model
# col_positive = ['HeadPoseAngles_Z', 'EyeState_Right', 'FaceLandmarks_17_X', 'FaceLandmarks_16_X', 'RightEyeBoundingBox_X', 
#                'EyeState_Left']

## 負相關特徵
col_negative = []
### 6pos6neg_model
# col_negative = ['FaceBoundingBox_Y', 'GazeVector_Z', 'HeadPoseAngles_Y', 'FaceLandmarks_14_Y', 'LeftEyeBoundingBox_Y', 
#                'FaceLandmarks_13_Y']
# col_negative = ['FaceBoundingBox_Y', 'GazeVector_Z', 'HeadPoseAngles_Y', 'FaceLandmarks_14_Y', 'LeftEyeBoundingBox_Y', 
#                'FaceLandmarks_13_Y', 'FaceLandmarks_15_Y', 'FaceLandmarks_16_Y', 'FaceLandmarks_19_Y', 'FaceLandmarks_17_Y', 
#                'FaceLandmarks_2_Y', 'EyeLandmarks_2_Y', 'LeftEyeMidPoint_Y', 'RightEyeBoundingBox_Y']

## 所有重要特徵－正相關特徵 + 負相關特徵
col_reserved = []
for col_name in col_positive:
    col_reserved.append(col_name)
for col_name in col_negative:
    col_reserved.append(col_name)

print(col_reserved)

# 刪除沒有被列在重要特徵列表的特徵。
def ReserveImportColumns(df_data, col_import):
    for col_name in df_data.columns.to_list():
        if col_name not in col_import:
            df_data = df_data.drop(col_name, axis = 1)
    return df_data

['HeadPoseAngles_Y', 'GazeVector_X', 'GazeVector_Z', 'GazeVector_Y', 'HeadPoseAngles_Z']


In [4]:
# Test
# 測試一個答案全部為1的資料。
df_test = pd.read_csv('/project/xt121-group5/scene2_data/test_data/2023-05-06 135830_c.csv')
df_test = FaceLandmarksPreprocessing(df_test)
df_test = ReserveImportColumns(df_test, col_reserved)

npa_test = df_test.to_numpy()

In [5]:
pred = model_xgb.predict(npa_test)
print(pred)

[0.98967814 0.98909724 0.9898996  0.9898996  0.9886093  0.98909724
 0.9865804  0.98944724 0.98909724 0.9865804  0.9865804  0.98909724
 0.98909724 0.98909724 0.9865804  0.9872937  0.9872937  0.98967814
 0.98909724 0.98921597 0.98967814 0.9872937  0.98909724 0.98967814
 0.98909724 0.9872937  0.9886093  0.9886093  0.9886093  0.9876659
 0.9872937  0.9875656  0.9881633  0.98909724 0.98967814 0.9872937
 0.9898996  0.98967814 0.98909724 0.98909724 0.9881633  0.9881633
 0.98909724 0.9881633  0.9872937  0.98944724 0.9881633  0.9898996
 0.98909724 0.98967814 0.98967814 0.98967814 0.98967814 0.98967814
 0.98967814 0.98967814 0.98967814 0.98967814 0.9795141  0.9852889
 0.98967814 0.98921597 0.98967814 0.98921597 0.98967814 0.98967814
 0.98967814 0.98967814 0.9898996  0.9852889  0.9883222  0.9865804
 0.98921597 0.98967814 0.98909724 0.98909724 0.9872937  0.9872937
 0.9872937  0.9872937  0.9795141  0.9872937  0.9872937  0.98967814
 0.9872937  0.98967814 0.98921597 0.9872937  0.98921597 0.98967814
 0

In [6]:
# Test
# 測試一個答案全部為0的資料。
df_test = pd.read_csv('/project/xt121-group5/scene2_data/test_data/2023-05-06 135847_nc.csv')
df_test = FaceLandmarksPreprocessing(df_test)
df_test = ReserveImportColumns(df_test, col_reserved)

npa_test = df_test.to_numpy()
pred = model_xgb.predict(npa_test)
print(pred)

[7.8109914e-01 7.8109914e-01 4.8341340e-01 7.8109914e-01 6.8279213e-01
 6.9796896e-01 8.0258614e-01 7.4860126e-01 7.9891568e-01 7.4611604e-01
 5.8738369e-01 5.7253915e-01 8.2414228e-01 1.5495272e-02 7.4166581e-02
 5.7215017e-01 1.5495272e-02 4.0918298e-02 6.4417219e-01 2.0133967e-02
 1.2348074e-02 1.2348074e-02 7.0468193e-01 3.5959798e-01 3.4189314e-01
 5.6418455e-01 2.0121711e-01 3.9733914e-01 1.3338348e-03 3.4048295e-01
 1.6779280e-01 1.6779280e-01 4.5375603e-01 1.2471709e-01 1.2471709e-01
 1.6779280e-01 1.6779280e-01 2.8145871e-01 1.6779280e-01 3.3025679e-01
 4.6061489e-01 3.3025679e-01 2.9590276e-01 2.6416814e-01 2.8768688e-01
 1.3856094e-01 1.6839005e-01 4.5228552e-02 1.6779280e-01 1.8209121e-01
 1.6779280e-01 1.6779280e-01 1.6779280e-01 4.5228552e-02 1.6779280e-01
 1.6779280e-01 4.5228552e-02 6.5849289e-02 6.9172555e-03 1.4705222e-02
 6.9172555e-03 1.4705222e-02 1.4705222e-02 1.3548027e-02 1.3548027e-02
 1.5913639e-03 1.4705222e-02 1.4705222e-02 1.7290793e-03 1.7290793e-03
 1.470

In [7]:
# Test larry
# 測試一個答案全部為1的資料。
df_test = pd.read_csv('/project/xt121-group5/scene2_data/test_data/2023-05-13 141120_c.csv')
df_test = FaceLandmarksPreprocessing(df_test)
df_test = ReserveImportColumns(df_test, col_reserved)

npa_test = df_test.to_numpy()
pred = model_xgb.predict(npa_test)
print(pred)

[0.8217612  0.8990702  0.8990702  0.9171296  0.924235   0.9210055
 0.9210055  0.91793734 0.9210055  0.9210055  0.9024746  0.85234
 0.9024746  0.9024746  0.86512417 0.86512417 0.8857591  0.85234
 0.85234    0.86512417 0.86512417 0.89279205 0.85234    0.9024746
 0.9024746  0.9024746  0.8813779  0.85234    0.86512417 0.86512417
 0.86512417 0.86512417 0.9024746  0.8919665  0.9024746  0.9024746
 0.9024746  0.89279205 0.9024746  0.9024746  0.9024746  0.9024746
 0.9024746  0.86512417 0.89279205 0.85234    0.89279205 0.89279205
 0.89279205 0.9024746  0.86512417 0.9024746  0.89279205 0.86512417
 0.9024746  0.85234    0.84861237 0.40602857 0.40602857 0.40602857
 0.33071768 0.08241176 0.55604345 0.8877007  0.89279205 0.89279205
 0.89279205 0.89279205 0.89279205 0.89279205 0.89279205 0.89279205
 0.86512417 0.9024746  0.9024746  0.9304809  0.9304809  0.9304809
 0.9304809  0.9428997  0.9304809  0.9304809  0.9304809  0.9304809
 0.9304809  0.9304809  0.9304809  0.9304809  0.9304809  0.9304809
 0.93048

In [8]:
pd.DataFrame(pred).describe()

,0
count,350.000000
mean,0.774087
std,0.251209
min,0.058854
25%,0.773011
50%,0.892792
75%,0.930481
max,0.949339


In [9]:
# Test larry
# 測試一個答案全部為0的資料。
df_test = pd.read_csv('/project/xt121-group5/scene2_data/test_data/2023-05-13 141134_nc.csv')
df_test = FaceLandmarksPreprocessing(df_test)
df_test = ReserveImportColumns(df_test, col_reserved)

npa_test = df_test.to_numpy()
pred = model_xgb.predict(npa_test)
print(pred)

[0.40602857 0.33071768 0.23336169 0.20940854 0.23336169 0.0116198
 0.08928756 0.03109748 0.07582808 0.02598812 0.01296898 0.0923342
 0.06435107 0.53178805 0.4524467  0.5444383  0.40602857 0.5444383
 0.5444383  0.5444383  0.5250718  0.5444383  0.5444383  0.5444383
 0.5444383  0.46273193 0.5444383  0.46273193 0.46273193 0.46273193
 0.46273193 0.46273193 0.5444383  0.5444383  0.40602857 0.40602857
 0.30207613 0.30207613 0.23831062 0.40602857 0.23831062 0.30207613
 0.23831062 0.30207613 0.30207613 0.23831062 0.30207613 0.5444383
 0.5444383  0.5444383  0.5444383  0.40602857 0.46273193 0.71842617
 0.6396835  0.7376959  0.61666006 0.8269232  0.8269232  0.868932
 0.8035139  0.827336   0.8692605  0.8035139  0.7673237  0.7673237
 0.7673237  0.8035139  0.7409918  0.8035139  0.84811133 0.7409918
 0.7409918  0.6453521  0.7038477  0.7673237  0.5571634  0.6206949
 0.64717007 0.74665105 0.8035139  0.6206949  0.52077615 0.30576625
 0.6206949  0.64717007 0.7005192  0.7005192  0.7005192  0.73873216
 0.72

In [10]:
pd.DataFrame(pred).describe()

,0
count,399.000000
mean,0.294957
std,0.234194
min,0.001267
25%,0.071179
50%,0.302076
75%,0.430748
max,0.869260


In [11]:
# Test PX
# 測試一個答案全部為1的資料。
df_test = pd.read_csv('/project/xt121-group5/scene2_data_01equal/pinsian/2023-05-13 142553_c.csv')
df_test = FaceLandmarksPreprocessing(df_test)
df_test = ReserveImportColumns(df_test, col_reserved)

npa_test = df_test.to_numpy()
pred = model_xgb.predict(npa_test)
print(pred)
print(pd.DataFrame(pred).describe())

[0.9576617  0.9576617  0.9576617  0.9576617  0.9576617  0.9576617
 0.9576617  0.9576617  0.9576617  0.928085   0.9576617  0.9576617
 0.9576617  0.91762114 0.91762114 0.90405357 0.91762114 0.91762114
 0.91762114 0.91086274 0.91762114 0.91762114 0.91762114 0.91086274
 0.91086274 0.91762114 0.928085   0.91095    0.9126508  0.9126508
 0.91086274 0.91086274 0.928085   0.9093051  0.9093051  0.9576617
 0.9576617  0.9576617  0.9576617  0.9576617  0.9576617  0.9576617
 0.9576617  0.9576617  0.9576617  0.9576617  0.91762114 0.91762114
 0.91762114 0.8434456  0.907367   0.91762114 0.907367   0.907367
 0.907367   0.907367   0.8315754  0.91437185 0.91437185 0.91762114
 0.928085   0.95591587 0.9576617  0.87651455 0.9576617  0.9576617
 0.9576617  0.928085   0.928085   0.928085   0.907367   0.9133399
 0.9133399  0.9133399  0.9133399  0.9065296  0.92795527 0.92795527
 0.92795527 0.9188737  0.92795527 0.92795527 0.9402576  0.9188737
 0.92795527 0.9188737  0.9402576  0.9402576  0.9318261  0.9402576
 0.918

In [12]:
# Test PX
# 測試一個答案全部為0的資料。
df_test = pd.read_csv('/project/xt121-group5/scene2_data_01equal/pinsian/2023-05-13 142611 nc.csv')
df_test = FaceLandmarksPreprocessing(df_test)
df_test = ReserveImportColumns(df_test, col_reserved)

npa_test = df_test.to_numpy()
pred = model_xgb.predict(npa_test)
print(pred)
print(pd.DataFrame(pred).describe())

[7.08055317e-01 5.05176485e-01 5.20778537e-01 5.57654500e-01
 5.20778537e-01 5.18017590e-01 4.34227616e-01 6.13390505e-01
 1.22471057e-01 1.13500334e-01 1.26479259e-02 6.41879626e-03
 5.30696707e-03 5.30696707e-03 5.30696707e-03 5.30696707e-03
 5.30696707e-03 6.44451985e-03 6.44451985e-03 6.44451985e-03
 5.30696707e-03 6.44451985e-03 5.28575992e-03 5.28575992e-03
 5.28575992e-03 5.74166793e-03 9.21018014e-04 9.21018014e-04
 9.18201928e-04 9.12127784e-04 5.73260011e-04 2.53641629e-03
 1.99877378e-03 1.40241743e-03 1.40241743e-03 1.96138606e-03
 1.96138606e-03 1.27047731e-03 1.33052084e-03 1.56759471e-03
 1.35887531e-03 1.35887531e-03 1.33052084e-03 1.33052084e-03
 1.96138606e-03 1.96138606e-03 1.33052084e-03 1.96138606e-03
 1.33052084e-03 3.54029093e-04 3.54029093e-04 4.17183095e-04
 3.63633910e-04 3.63633910e-04 3.62658000e-04 3.63633910e-04
 1.36546418e-03 3.63336236e-04 3.63336236e-04 3.63336236e-04
 3.63336236e-04 3.53121693e-04 3.53121693e-04 3.63633910e-04
 3.62658000e-04 3.531216